In [23]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import psycopg2

In [24]:
POSTGRES_USER="ariel"
POSTGRES_PASS="ariel"
POSTGRES_HOST="localhost"
POSTGRES_DB="eco_bikes"
POSTGRES_PORT="5432"
POSTGRES_SCHEMA="eco_bikes"
DB_STR = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASS}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

In [25]:
conn = create_engine(DB_STR, connect_args={'options': f'-csearch_path={POSTGRES_SCHEMA}'})

In [26]:
data_scr=pd.read_parquet("/tmp/station_info_eco_bikes.parquet")

In [27]:
data_scr.drop(columns="reload_id",inplace=True)

In [28]:
data_scr.columns

Index(['station_id', 'station_name', 'physical_configuration', 'lat', 'lon',
       'altitude', 'address', 'capacity', 'is_charging_station',
       'nearby_distance', '_ride_code_support'],
      dtype='object')

In [29]:
data_target=pd.read_sql_query("SELECT * FROM eco_bikes.station_info_eco_bikes",con=conn)
data_target.drop(columns="reload_id",inplace=True)

In [30]:
data_target

,pk_surrogate_station_info,station_id,station_name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station,nearby_distance,_ride_code_support,start_date,end_date,is_active
0,1,2,002 - Retiro I,SMARTLITMAPFRAME,-34.592424,-58.374710,0.0,AV. Dr. José María Ramos Mejía 1300,40,False,1000.0,True,2023-10-26 18:34:50.099763,9999-12-30 00:00:00,1
1,3,4,004 - Plaza Roma,SMARTLITMAPFRAME,-34.603008,-58.368856,0.0,Av. Corrientes 100,20,False,1000.0,True,2023-10-26 18:34:50.099763,9999-12-30 00:00:00,1
2,4,5,005 - Plaza Italia,SMARTLITMAPFRAME,-34.580550,-58.420954,0.0,Av. Sarmiento 2601,42,False,1000.0,True,2023-10-26 18:34:50.099763,9999-12-30 00:00:00,1
3,5,6,006 - Parque Lezama,SMARTLITMAPFRAME,-34.628526,-58.369758,0.0,"Avenida Martin Garcia, 295",20,False,1000.0,True,2023-10-26 18:34:50.099763,9999-12-30 00:00:00,1
4,6,7,007 - OBELISCO,SMARTLITMAPFRAME,-34.606498,-58.381098,0.0,CARLOS PELEGRINI 215,24,False,1000.0,True,2023-10-26 18:34:50.099763,9999-12-30 00:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,348,533,077 - CRISOLOGO LARRALDE,SMARTLITMAPFRAME,-34.561191,-58.492030,0.0,Galván 3601,20,False,1000.0,True,2023-10-26 18:34:50.099763,9999-12-30 00:00:00,1
347,349,534,081 - PARQUE DE LA CIENCIA,SMARTLITMAPFRAME,-34.582963,-58.429979,0.0,Soler 5300,24,False,1000.0,True,2023-10-26 18:34:50.099763,9999-12-30 00:00:00,1
348,350,535,090 - HONORIO PUEYRREDON,SMARTLITMAPFRAME,-34.602517,-58.449659,0.0,Honorio Pueyrredón 1653,20,False,1000.0,True,2023-10-26 18:34:50.099763,9999-12-30 00:00:00,1
349,351,536,019 - PLAZA SAN MARTIN,SMARTLITMAPFRAME,-34.595474,-58.377482,0.0,Av. Santa Fe 716,20,False,1000.0,True,2023-10-26 18:34:50.099763,9999-12-30 00:00:00,1


## CODE TO MAKE SCD

In [31]:
datetime_now=pd.to_datetime(dt.datetime.now(),format="%Y-%m-%d %HH:%MM:%SS")

In [32]:
data_target_current=data_target[(data_target["is_active"]==1)]

In [33]:
df_merge_col=pd.merge(data_scr,data_target_current,on='station_id',how='left')

In [34]:
new_records_filter=pd.isnull(df_merge_col).any(axis=1)


In [35]:
df_new_records = df_merge_col[new_records_filter]

In [36]:
df_excluding_new = pd.concat([df_merge_col, df_new_records],axis=0).drop_duplicates(keep=False)

In [37]:
df_new_records_final=df_new_records.copy()
df_new_records_final=df_new_records_final[['station_id',"station_name_x","physical_configuration_x","lat_x","lon_x","altitude_x","address_x","capacity_x","is_charging_station_x","nearby_distance_x","_ride_code_support_x"]]
df_new_records_final.columns=['station_id', 'station_name', 'physical_configuration', 'lat', 'lon','altitude', 'address', 'capacity', 'is_charging_station','nearby_distance', '_ride_code_support']

In [38]:
df_new_records_final['start_date']=datetime_now
df_new_records_final['end_date']="9999-12-30 00:00:00"
df_new_records_final['is_active']=1
df_new_records_final['reload_id']=1

In [39]:
df_scd2_records=df_excluding_new[(df_excluding_new["station_name_x"]!=df_excluding_new["station_name_y"] ) | 
                                 (df_excluding_new["physical_configuration_x"]!=df_excluding_new["physical_configuration_y"]) | 
                                 (df_excluding_new["lat_x"]!=df_excluding_new["lat_y"]) | 
                                 (df_excluding_new["lon_x"]!=df_excluding_new["lon_y"]) | 
                                 (df_excluding_new["altitude_x"]!=df_excluding_new["altitude_y"]) | 
                                 (df_excluding_new["address_x"]!=df_excluding_new["address_y"]) | 
                                 (df_excluding_new["capacity_x"]!=df_excluding_new["capacity_y"]) | 
                                 (df_excluding_new["is_charging_station_x"]!=df_excluding_new["is_charging_station_y"]) | 
                                 (df_excluding_new["nearby_distance_x"]!=df_excluding_new["nearby_distance_y"]) | 
                                 (df_excluding_new["_ride_code_support_x"]!=df_excluding_new["_ride_code_support_y"])]

In [40]:
df_scd2_records_final_replace=df_scd2_records.copy()
df_scd2_records_final_replace=df_scd2_records_final_replace[['station_id',"station_name_y","physical_configuration_y","lat_y","lon_y","altitude_y","address_y","capacity_y","is_charging_station_y","nearby_distance_y","_ride_code_support_y","start_date"]]
df_scd2_records_final_replace.columns=['station_id', 'station_name', 'physical_configuration', 'lat', 'lon','altitude', 'address', 'capacity', 'is_charging_station','nearby_distance', '_ride_code_support',"start_date"]
df_scd2_records_final_replace['start_date']=df_scd2_records_final_replace['start_date'].astype(str)
df_scd2_records_final_replace['end_date']=datetime_now
df_scd2_records_final_replace['is_active']=0
df_scd2_records_final_replace

,station_id,station_name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station,nearby_distance,_ride_code_support,start_date,end_date,is_active
1,3,003 - ADUANA,SMARTLITMAPFRAME,-34.611102,-58.368174,0.0,Moreno & Azopardo,1000,False,1000.0,True,2023-10-26 18:34:50.099763,2023-10-26 18:35:42.808431,0


In [41]:
df_scd2_records_final_append=df_scd2_records.copy()
df_scd2_records_final_append=df_scd2_records_final_append[['station_id',"station_name_x","physical_configuration_x","lat_x","lon_x","altitude_x","address_x","capacity_x","is_charging_station_x","nearby_distance_x","_ride_code_support_x"]]
df_scd2_records_final_append.columns=['station_id', 'station_name', 'physical_configuration', 'lat', 'lon','altitude', 'address', 'capacity', 'is_charging_station','nearby_distance', '_ride_code_support']
df_scd2_records_final_append['start_date']=datetime_now
df_scd2_records_final_append['end_date']="9999-12-30 00:00:00"
df_scd2_records_final_append['is_active']=1
df_scd2_records_final_append['reload_id']=1
df_scd2_records_final_append

,station_id,station_name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station,nearby_distance,_ride_code_support,start_date,end_date,is_active,reload_id
1,3,003 - ADUANA,SMARTLITMAPFRAME,-34.611102,-58.368174,0.0,Moreno & Azopardo,27,False,1000.0,True,2023-10-26 18:35:42.808431,9999-12-30 00:00:00,1,1


# LOAD TO DATABSE

In [42]:
try:
    connection = psycopg2.connect(user="ariel",
                                  password="ariel",
                                  host="localhost",
                                  port="5432",
                                  database="eco_bikes")
    cur=connection.cursor()
    for index, row in df_scd2_records_final_replace.iterrows():
        # Assuming your_table_name is the name of the table you want to update
        # Assuming your_primary_key_column is the primary key column of your table
        # Assuming your_primary_key_value is the value of the primary key for the specific row you want to update
        update_query = f"UPDATE eco_bikes.station_info_eco_bikes SET "
        
        # Dynamically construct the SET clause of the update query
        set_clauses = ', '.join([f"{col} = %s" for col in df_scd2_records_final_replace.columns])
        update_query += set_clauses
        
        # Specify the WHERE clause for the specific row to update
        update_query += f" WHERE station_id = '{row['station_id']}' and is_active=1 "
        
        # Extract values from the DataFrame
        values = tuple([row[col] for col in df_scd2_records_final_replace.columns])
               
        # Execute the update query with parameterized values
        cur.execute(update_query, values)
        connection.commit()
except (Exception, psycopg2.Error) as error:
    print("Error while fetching data from PostgreSQL", error)

finally:
    cur.close()
    connection.close()

In [43]:
df_new_records_final.to_sql('station_info_eco_bikes',index=False,con=conn,if_exists="append")

0

In [44]:
df_scd2_records_final_append.to_sql('station_info_eco_bikes',index=False,con=conn,if_exists="append")

1